In [29]:
# 对静态人脸图像文件进行68个特征点的标定
import dlib  # 人脸识别的库 Dlib
import numpy as np,numpy  # 数据处理的库 numpy
import cv2   # 图像处理的库 OpenCv
import sys

FACE_POINTS = list(range(17, 68))
MOUTH_POINTS = list(range(48, 61))
RIGHT_BROW_POINTS = list(range(17, 22))
LEFT_BROW_POINTS = list(range(22, 27))
RIGHT_EYE_POINTS = list(range(36, 42))
LEFT_EYE_POINTS = list(range(42, 48))
NOSE_POINTS = list(range(27, 35))
JAW_POINTS = list(range(0, 17))

OVERLAY_POINTS = [JAW_POINTS+LEFT_BROW_POINTS+RIGHT_BROW_POINTS,]

In [30]:
# Dlib 检测器和预测器
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

#获取68个面孔特征点，输入函数是图片，返回68个图片的像素点数组，数组index从0开始
def get_landmarks(img):
    faces = detector(img, 1)# 人脸数，第二个参数inclass为1时检测RGB图片，参数为0时检测灰度图片
    if len(faces) == 1:
        return np.matrix([[p.x, p.y] for p in predictor(img, faces[0]).parts()])
    raise NoFaceOrTooManyFaces

In [31]:
# 从text文本中读取特征点
def readPoints(path) :
    # Create an array of points.
    points = [];
    # Read points
    with open(path) as file :
        for line in file :
            x, y = line.split()
            points.append((int(x), int(y)))

    return points

# Apply affine transform calculated using srcTri and dstTri to src and
# output an image of size.
def applyAffineTransform(src, srcTri, dstTri, size) :
    #print('--------')
    #print(srcTri)
    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform( np.float32(srcTri), np.float32(dstTri) )
    # Apply the Affine Transform just found to the src image
    #像素使用线性插值的方式来对RGB颜色通道进行融合，flags=cv2.INTER_LINEAR
    dst = cv2.warpAffine( src, warpMat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )
    return dst

# Warps and alpha blends triangular regions from img1 and img2 to img
# 参数t为list类型
def morphTriangle(img1, img2, img, t1, t2, t, alpha) :
    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t]))
    #print('merge-r=',r)
    # Offset points by left top corner of the respective rectangles
    t1Rect = []
    t2Rect = []
    tRect = []
    
    #这里的t1 t2 t里存放的都是面部特征点的index
    for i in range(0, 3):
        tRect.append(((t[i][0] - r[0]),(t[i][1] - r[1])))
        t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2Rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))
    # Get mask by filling triangle
    mask = np.zeros((r[3], r[2], 3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(tRect), (1.0, 1.0, 1.0), 16, 0);
    # Apply warpImage to small rectangular patches
    img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    img2Rect = img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]]
    size = (r[2], r[3])
    #print('merge-shape=',img1Rect.shape)
    warpImage1 = applyAffineTransform(img1Rect, t1Rect, tRect, size)
    warpImage2 = applyAffineTransform(img2Rect, t2Rect, tRect, size)
    
    # Alpha blend rectangular patches
    imgRect = (1.0 - alpha) * warpImage1 + alpha * warpImage2

    # Copy triangular region of the rectangular patch to the output image
    img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] = img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] * ( 1 - mask ) + imgRect * mask

In [32]:
#利用三角仿射把被融合图片的脸部轮廓、关键点变形为上面得到的脸部关键点
#根据morph得到的特征点把模板图的脸部做变形
def tranSrc(src_img, src_points, dst_points):
    
    r = cv2.boundingRect(np.float32(dst_points))
    center = (r[0] + int(r[2] / 2), r[1] + int(r[3] / 2)) 
    srcTri=[]
    dstTri=[]
    #选取脸部轮廓特征点0-26做变形即可
    for i in range(0,34):
        srcTri.append(src_points[i])
        srcTri.append(src_points[i+1])
        srcTri.append((center[0],center[1]))
        dstTri.append(dst_points[i])
        dstTri.append(dst_points[i+1])
        dstTri.append((center[0],center[1]))
        #将模板图特征点和中心点组成三角形并获取矩形
        t = [src_points[i],src_points[i+1],(center[0],center[1])]
        r = cv2.boundingRect(np.float32(t))
        #print('r=',r)
        #切割出三角形的像素和坐标
        img = src_img[r[1]:r[1] + r[3], r[0]:r[0] + r[2]]
        size = (r[2], r[3])
        # Given a pair of triangles, find the affine transform.
        #print('++++++++++')
        #print(i,'r=',r,'t=',t)
        #print(i,'size0=',size[0],'size1=',size[1],'img-shape=',img.shape)
        #print(i,'srcTri=',srcTri,'dstTri=',dstTri)
        warpMat = cv2.getAffineTransform(np.float32(srcTri), np.float32(dstTri))
        #清空list防止参数过多
        srcTri.clear()
        dstTri.clear()
        #print(i,'warp=',warpMat)
        #Apply the Affine Transform just found to the src image
        #像素使用线性插值的方式来对RGB颜色通道进行融合，flags=cv2.INTER_LINEAR
        #print(i,'src_img',src_img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]])
        src_img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]]= cv2.warpAffine(img, warpMat,(size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )
        #print(i,'Affine=',src_img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]])

In [39]:
#dst_img = merge_img(src_img, dst_img, dst_matrix, dst_points, k_size, mat_multiple)
#最后一步是将融合后的新图片脸部区域用泊松融合算法贴到模特图上。泊松融合可直接使用opencv提供的函数
#src_img是模板图 dst_img是融合后要贴入的图 src_poing 和dst_point是融合后的图特征点的矩阵和数组类型
#mat_multiple —— 缩放获取到的人脸心型区域大小默认0.95
#blur_size —— 模糊核大小，用于模糊人脸融合边缘，减少融合后的违和感,默认(15,10)
#face_mask —— 是子图的掩图
#center —— 表示坐标，你打算在母图的哪个位置放子图
#cv2.NORMAL_CLONE代表融合的模式
def merge_img(src_img, dst_img, src_points, dst_points, k_size=None, mat_multiple=None):
    
    face_mask = np.zeros(dst_img.shape, dtype=dst_img.dtype) 
    dst_points=np.array(dst_points)
    for group in OVERLAY_POINTS:
        cv2.fillConvexPoly(face_mask, cv2.convexHull(np.int32(dst_points[group])), (255, 255, 255))
    # Display Result
    #cv2.imshow("Morphed Face",np.uint8(face_mask))
    #cv2.waitKey(0)    
    #center报错
    r = cv2.boundingRect(np.float32([src_points[:16]]))
    center = (r[0] + int(r[2] / 2), r[1] + int(r[3] / 2))
#     width, height, channels = src_img.shape
#     center = (height//2, width//2)
    
    if mat_multiple:
        mat = cv2.getRotationMatrix2D(center, 0, mat_multiple)
        face_mask = cv2.warpAffine(face_mask, mat, (face_mask.shape[1], face_mask.shape[0]))    
    if k_size:
        face_mask = cv2.blur(face_mask, k_size, center)
        #cv2.seamlessClone(src, dst, mask, center, flags)
        #src:将要被克隆到目标图上的原图，融合后的面孔。
        #dst:目标图，模板图。
        #mask：想要克隆的目标图的mask。mask的大小要与原图大小一致。
        #center ：目标影像的中心在背景图像上的坐标！注意是目标影像的中心
        #flags： cv2.NORMAL_CLONE and cv2.MIXED_CLONE. 
        #cv2.seamlessClone返回图片 
    return cv2.seamlessClone(dst_img, src_img, face_mask, center, cv2.NORMAL_CLONE)

In [40]:
if __name__=='__main__':
    filename1 = '/home/fengchao/桌面/11.png'
    filename2 = '/home/fengchao/桌面/3.png'
    alpha = 0.5 #alpha调节融合程度，数值越小第一张图融合成分越多
    
    # 读取图片
    img1 = cv2.imread(filename1);
    img2 = cv2.imread(filename2);

    # 将图片像素值转换成浮点数
    img1 = np.float64(img1)
    img2 = np.float64(img2)
    # 读取两张图片的特征点
    points1 = readPoints(filename1 + '.txt')
    points2 = readPoints(filename2 + '.txt')

    #创建一个list存放morph后的特征点
    morph_points = []
    #计算alpha权重特征点获取融合特征点
    for i in range(0, len(points1)):
        x = ( 1 - alpha ) * points1[i][0] + alpha * points2[i][0]
        y = ( 1 - alpha ) * points1[i][1] + alpha * points2[i][1]
        morph_points.append((x,y))
        
    #为  模板面孔赋值用于生成基于图片一的图片空间
    morph_img= np.zeros(img1.shape,dtype = img1.dtype)
    src_img= np.zeros(img1.shape,dtype = img1.dtype)
    # 从tri.txt文档中读入面部三角
    with open("tri.txt") as file :
        for line in file :
            x,y,z = line.split()
            
            x = int(x)
            y = int(y)
            z = int(z)
            
            t1 = [points1[x], points1[y], points1[z]]
            t2 = [points2[x], points2[y], points2[z]]
            t = [morph_points[x], morph_points[y], morph_points[z]]
            #三角融合算法，对上一步转换后的待融合图片再次取关键点，然后与模特图的关键点一起做三角融合成新的图片
            morphTriangle(img1, img2, morph_img, t1, t2, t, alpha)     
    #对模板进行三角仿射变形
    #src_img=img1
    #tranSrc(src_img, points1, morph_points)
    cv2.imwrite('/home/fengchao/桌面/morph_img.jpg',morph_img)
    #泊松融合
    morph_points_matrix= np.mat(morph_points)
    dst = merge_img(img1, morph_img, points1, morph_points,(15,10),0.95)

    # Display Result
    #cv2.imshow("Morphed Face",np.uint8(dst_img))
    #cv2.waitKey(0)
    

error: /feedstock_root/build_artefacts/opencv_1513957419294/work/opencv-3.3.0/modules/imgproc/src/color.cpp:10600: error: (-215) depth == CV_8U || depth == CV_16U || depth == CV_32F in function cvtColor


In [ ]:
    # Read images : src image will be cloned into dst
    im = cv2.imread('/home/fengchao/桌面/11.png')
    obj= cv2.imread('/home/fengchao/桌面/morph_img.jpg')

    # Create an all white mask
    #mask = 255 * np.ones(obj.shape, obj.dtype)
    face_mask = np.zeros(obj.shape, dtype=obj.dtype) 
    print(face_mask.shape)
    dst_points=np.array(morph_points)
    for group in OVERLAY_POINTS:
        cv2.fillConvexPoly(face_mask, cv2.convexHull(np.int32(dst_points[group])), (255, 255, 255))
    print(face_mask.shape)
    # The location of the center of the src in the dst
    width, height, channels = im.shape
    center = (height//2, width//2)
    r = cv2.boundingRect(np.float32([points1[:16]]))
    center1 = (r[0] + int(r[2] / 2), r[1] + int(r[3] / 2))
    
    print (center)
    print (center1)
    # Seamlessly clone src into dst and put the results in output
    normal_clone = cv2.seamlessClone(obj, im, mask, center, cv2.NORMAL_CLONE)

    # Write results
    cv2.imwrite("opencv-normal-clone-example.jpg", normal_clone)
    

In [ ]:
#最后一步是将融合后的新图片脸部区域用泊松融合算法贴到模特图上。泊松融合可直接使用opencv提供的函数
#src_img是模板图 dst_img是融合后要贴入的图 dst_matrix 和dst_point是融合后的图特征点的矩阵和数组类型
#mat_multiple —— 缩放获取到的人脸心型区域大小默认0.95
#blur_size —— 模糊核大小，用于模糊人脸融合边缘，减少融合后的违和感,默认(15,10)
#face_mask —— 是子图的掩图
#center —— 表示坐标，你打算在母图的哪个位置放子图
#cv2.NORMAL_CLONE代表融合的模式
def merge_img(src_img, dst_img, dst_points):
    
    face_mask = 255*np.ones(dst_points.shape, dtype=dst_img.dtype)    
    cv2.imshow("Morphed Face",np.uint8(face_mask))
    cv2.waitKey(0)
    dst_points=np.int64(dst_points)
    #cv2.fillConvexPoly(face_mask, dst_points, (255, 255, 255))
    #cv2.fillConvexPoly(face_mask, dst_points, (255, 255, 255))
        
    r = cv2.boundingRect(np.float32(dst_points))
    center = (r[0] + int(r[2] / 2), r[1] + int(r[3] / 2)) 
        #cv2.seamlessClone(src, dst, mask, center, flags)
        #src:将要被克隆到目标图上的原图，融合后的面孔。
        #dst:目标图，模板图。
        #mask：想要克隆的目标图的mask。mask的大小要与原图大小一致。
        #center ： 在目标图中原图的中心位置。
        #flags：  cv2.NORMAL_CLONE and cv2.MIXED_CLONE. 
        #cv2.seamlessClone返回图片
    return cv2.seamlessClone(np.uint8(dst_img), src_img, face_mask, center, cv2.NORMAL_CLONE)